In [9]:
prompt = """
Label each token in the sentence below with the appropriate BIO entity tag.
Use these labels:
{{1: 'B-geo', 2: 'B-gpe', 3: 'B-org', 4: 'B-per', 5: 'B-tim', 6: 'I-geo', 7: 'I-org', 8: 'I-per', 9: 'I-tim', 0: 'O'}}

Example:
Sentence: Paris is a city in France.
Labels (in JSON):
[
  {{"token": "Paris", "tag": "B-geo"}},
  {{"token": "is", "tag": "O"}},
  {{"token": "a", "tag": "O"}},
  {{"token": "city", "tag": "O"}},
  {{"token": "in", "tag": "O"}},
  {{"token": "France", "tag": "B-geo"}},
  {{"token": ".", "tag": "O"}}
]

Now label this sentence:
Sentence: {sentence}

Labels (in JSON):
"""


In [10]:
sentence = "channdeth"

In [11]:
def generateNER(sentence, llm):
    response = llm.invoke(prompt.format(sentence=sentence))
    return response.content

In [12]:
prompt.format(sentence=sentence)

'\nLabel each token in the sentence below with the appropriate BIO entity tag.\nUse these labels:\n{1: \'B-geo\', 2: \'B-gpe\', 3: \'B-org\', 4: \'B-per\', 5: \'B-tim\', 6: \'I-geo\', 7: \'I-org\', 8: \'I-per\', 9: \'I-tim\', 0: \'O\'}\n\nExample:\nSentence: Paris is a city in France.\nLabels (in JSON):\n[\n  {"token": "Paris", "tag": "B-geo"},\n  {"token": "is", "tag": "O"},\n  {"token": "a", "tag": "O"},\n  {"token": "city", "tag": "O"},\n  {"token": "in", "tag": "O"},\n  {"token": "France", "tag": "B-geo"},\n  {"token": ".", "tag": "O"}\n]\n\nNow label this sentence:\nSentence: channdeth\n\nLabels (in JSON):\n'

In [16]:
result = generateNER('Despite heavy rain and unexpected delays at Charles de Gaulle Airport, Dr. Angela Martinez, the renowned neuroscientist from the University of Barcelona, managed to arrive in Tokyo on time for the International Brain Research Symposium, where she presented her groundbreaking research to an audience of experts from around the world.',llm)

In [17]:
print(result)

[
  {"token": "Despite", "tag": "O"},
  {"token": "heavy", "tag": "O"},
  {"token": "rain", "tag": "O"},
  {"token": "and", "tag": "O"},
  {"token": "unexpected", "tag": "O"},
  {"token": "delays", "tag": "O"},
  {"token": "at", "tag": "O"},
  {"token": "Charles", "tag": "B-org"},
  {"token": "de", "tag": "I-org"},
  {"token": "Gaulle", "tag": "I-org"},
  {"token": "Airport", "tag": "I-org"},
  {"token": ",", "tag": "O"},
  {"token": "Dr.", "tag": "B-per"},
  {"token": "Angela", "tag": "I-per"},
  {"token": "Martinez", "tag": "I-per"},
  {"token": ",", "tag": "O"},
  {"token": "the", "tag": "O"},
  {"token": "renowned", "tag": "O"},
  {"token": "neuroscientist", "tag": "O"},
  {"token": "from", "tag": "O"},
  {"token": "the", "tag": "O"},
  {"token": "University", "tag": "B-org"},
  {"token": "of", "tag": "I-org"},
  {"token": "Barcelona", "tag": "I-org"},
  {"token": ",", "tag": "O"},
  {"token": "managed", "tag": "O"},
  {"token": "to", "tag": "O"},
  {"token": "arrive", "tag": "O"},

In [25]:
prompt_ds = """
Generate a synthetic dataset for Named Entity Recognition (NER) in BIO format.
For each row:
- Provide a complex sentence.
- Tokenize the sentence (split into tokens).
- Assign each token a BIO tag, choosing only from these options:
  'B-geo', 'B-gpe', 'B-org', 'B-per', 'B-tim', 'I-geo', 'I-org', 'I-per', 'I-tim', 'O'

Output a JSON array, where each item contains:
- "sentence": the full sentence,
- "tokens": a list of tokens,
- "tags": a list of corresponding BIO tags (as strings).

Generate {num_rows} such rows.

Example:
[
  {{
    "sentence": "Paris is a city in France.",
    "tokens": ["Paris", "is", "a", "city", "in", "France", "."],
    "tags": ["B-geo", "O", "O", "O", "O", "B-geo", "O"]
  }}
  // ... more rows
]

Now generate {num_rows} such rows.
"""


In [125]:
import re
import json
import random
import pandas as pd
from typing import Dict, Any, List
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
import spacy
from spacy.training import Example
from spacy.training import offsets_to_biluo_tags
from spacy.training import biluo_tags_to_offsets
from spacy.util import minibatch
import torch
from sklearn.model_selection import train_test_split
from pdb import set_trace
import random
import ast

In [ ]:
template="""
You are an expert in Named Entity Recognition (NER).

Generate {num_row} realistic training examples in the following JSON-safe format:
[
  ["Sentence with entities", ["BILUO_tag_1", "BILUO_tag_2", ...]],
  ...
]

Rules:
- Each sentence must be natural and realistic, resembling real-world contexts (e.g., IT reports, server logs, emails, network configurations).
- Use entity labels ONLY from this list: {entity_list}, plus mandatory labels "Token keys" and "IP-Address". Do NOT use other labels like "Token".
- Each sentence must include:
  - One "Token keys" entity: a random alphanumeric string (20-30 characters, e.g., "xY9zPqW3kL8mN2vT7rJ5hF6tQ"), labeled as "Token keys" (e.g., "U-Token keys" for single-token, or "B-Token keys", "I-Token keys", "L-Token keys" for multi-token).
  - One "IP-Address" entity: either IPv4 (e.g., "192.168.1.1") or IPv6 (e.g., "2001:0db8:85a3:0000:0000:8a2e:0370:7334"), labeled as "IP-Address" (e.g., "U-IP-Address").
  - At least one entity from {entity_list} (e.g., GPE, ORG, PERSON).
- BILUO tags must correspond to tokens produced by spaCy's en_core_web_sm tokenizer.
- Use "U-<label>" for single-token entities (e.g., "U-GPE", "U-Token keys", "U-IP-Address") and "B-<label>", "I-<label>", "L-<label>" for multi-token entities (e.g., "B-PERSON", "L-PERSON").
- Ensure tokens and BILUO tags align perfectly with spaCy's tokenizer to avoid misalignment errors.
- Use uppercase labels (e.g., "GPE", "ORG", "PERSON", "Token keys", "IP-Address") to match en_core_web_sm.
- Output must be valid JSON — use [ ] for BILUO tags, no extra explanation.
- Example format:
  [
    ["The German firm works as a sub-contractor for Shell .", ["O", "U-GPE", "O", "O", "O", "O", "O", "O", "U-ORG", "O"]],
    ["Server in Germany accessed with token xY9zPqW3kL8mN2vT7rJ5hF6tQ from IP 192.168.1.1.", ["O", "O", "U-GPE", "O", "O", "O", "U-Token keys", "O", "O", "U-IP-Address", "O"]],
    ...
  ]
Return a list of {num_row} examples in this exact format.
"""

In [425]:
import re
import json
import pandas as pd
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI


def clean_and_parse_llm_output(text: str):
    # Step 1: Basic cleaning
    text = text.replace('```python', '').replace('```', '').strip()
    text = text.replace('“', '"').replace('”', '"').replace("‘", "'").replace("’", "'")

    # Remove non-printable characters except newline/tab
    text = ''.join(c for c in text if c.isprintable() or c in '\n\t')

    # Step 2: Convert tuples to JSON-style lists using regex
    text = re.sub(r'\((\d+),\s*(\d+),\s*"([^"]+)"\)', r'[\1, \2, "\3"]', text)
    
    return text

def generate_ner_ds(num_row: int, model_name: str) -> pd.DataFrame:
    prompt = PromptTemplate(
        input_variables=["num_row"],
        template="""
Generate a synthetic dataset for Named Entity Recognition (NER) in BIO format.
For each row:
- Provide a complex sentence containing at least one 'Token' entity, one 'IP' entity, and one or more other entities (e.g., geo, gpe, org, per, tim).
- Tokenize the sentence (split into tokens, ensuring proper handling of punctuation and special characters).
- Assign each token a BIO tag from the following options:
  'B-geo', 'B-gpe', 'B-org', 'B-per', 'B-tim', 'I-geo', 'I-gpe', 'I-org', 'I-per', 'I-tim', 'B-Token', 'B-IP', 'O'
  Where:
  - 'Token': A random string of 15 to 50 characters containing alphanumeric characters (letters and digits) and optionally the special characters '_' or '-'. The string must not resemble a name, date, or other meaningful entity. Examples: '3tyslghiouyeuiosyet9w_euyi34-sfj', 'ab12_xy34-pqrs7890'.
  - 'IP': A valid IPv4 (e.g., '192.168.1.101') or IPv6 (e.g., '2001:0db8:85a3:0000:0000:8a2e:0370:7334') address.
- Ensure that multi-word entities (e.g., 'New York') are tagged with 'B-' for the first token and 'I-' for subsequent tokens.
- Include at least one 'Token' and one 'IP' entity in each sentence, along with at least one other entity (geo, gpe, org, per, tim).
- To avoid overfitting, vary the context of 'Token' entities:
  - Use different words or phrases before 'Token' entities, such as 'key', 'credential', 'code', or no word at all. Only 30% of sentences should use the word 'token' before a 'Token' entity.
  - Place 'Token' entities in varied syntactic positions (e.g., subject, object, within prepositional phrases).
  - Use diverse verbs and sentence structures (e.g., 'authenticated', 'verified', 'accessed', 'sent').

Output a list of dictionaries, where each dictionary contains:
- "sentence": the full sentence (string),
- "tokens": a list of tokens (list of strings),
- "tags": a list of corresponding BIO tags (list of strings).
- Rule: The length of 'tokens' must equal the length of 'tags'.

Example:
[
  {{
    "sentence": "Paris is a city in France.",
    "tokens": ["Paris", "is", "a", "city", "in", "France", "."],
    "tags": ["B-geo", "O", "O", "O", "O", "B-geo", "O"]
  }},
  {{
    "sentence": "The key x7y9z_kjhg23-iopqwe456 authenticated the user at 172.16.254.10 for Google.",
    "tokens": ["The", "key", "x7y9z_kjhg23-iopqwe456", "authenticated", "the", "user", "at", "172.16.254.10", "for", "Google", "."],
    "tags": ["O", "O", "B-Token", "O", "O", "O", "O", "B-IP", "O", "B-org", "O"]
  }},
  {{
    "sentence": "Elon Musk accessed the system from 2001:0db8:85a3:0000:0000:8a2e:0370:7334 on Monday.",
    "tokens": ["Elon", "Musk", "accessed", "the", "system", "from", "2001:0db8:85a3:0000:0000:8a2e:0370:7334", "on", "Monday", "."],
    "tags": ["B-per", "I-per", "O", "O", "O", "O", "B-IP", "O", "B-tim", "O"]
  }},
  {{
    "sentence": "Credential zxcv987_12pqr-456tuv was verified in Tokyo at 192.168.1.1.",
    "tokens": ["Credential", "zxcv987_12pqr-456tuv", "was", "verified", "in", "Tokyo", "at", "192.168.1.1", "."],
    "tags": ["O", "B-Token", "O", "O", "O", "B-geo", "O", "B-IP", "O"]
  }}
]

Generate {num_row} such rows.
"""
        
    )

    model = ChatOpenAI(model_name=model_name)
    chain = prompt | model
    response = chain.invoke({"num_row": num_row}).content
    
    parsed_data = clean_and_parse_llm_output(response)
    response = ast.literal_eval(parsed_data)
  
   
    # Extract into DataFrame
    rows = []
    for item in response:
        rows.append({
            "sentence": item["sentence"],
            "tokens": item["tokens"],
            "tags": item["tags"]
        })

    return pd.DataFrame(rows)


In [519]:
import re
import ast
import pandas as pd
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI

def clean_and_parse_llm_output(text: str):
    text = text.replace('```python', '').replace('```', '').strip()
    text = text.replace('“', '"').replace('”', '"').replace("‘", "'").replace("’", "'")
    text = ''.join(c for c in text if c.isprintable() or c in '\n\t')
    text = re.sub(r'\((\d+),\s*(\d+),\s*"([^"]+)"\)', r'[\1, \2, "\3"]', text)
    return text

def generate_ner_test_1(num_row: int, model_name: str) -> pd.DataFrame:
    prompt = PromptTemplate(
        input_variables=["num_row"],
        template="""
Generate a synthetic dataset for Named Entity Recognition (NER) in BIO format.
For each row:
- Provide a complex sentence containing at least one 'Token' entity, one 'IP' entity.
- Tokenize the sentence (split into tokens, ensuring proper handling of punctuation and special characters).
- Assign each token a BIO tag from the following options:
  'B-Token', 'B-IP', 'O'
  Where:
  - 'Token': A random string of 15 to 50 characters containing alphanumeric characters (letters and digits) and optionally the special characters '_' or '-'. The string must not resemble a name, date, or other meaningful entity. Examples: '3tyslghiouyeuiosyet9w_euyi34-sfj', 'ab12_xy34-pqrs7890'.
  - 'IP': A valid IPv4 (e.g., '192.168.1.101') or IPv6 (e.g., '2001:0db8:85a3:0000:0000:8a2e:0370:7334') address.
- To avoid overfitting, vary the context of 'Token' entities:
  - Place 'Token' entities in varied syntactic positions (e.g., subject, object, within prepositional phrases).
  - Avoid using entity type before the word "token".

Output a list of dictionaries, where each dictionary contains:
- "sentence": the full sentence (string),
- "tokens": a list of tokens (list of strings),
- "tags": a list of corresponding BIO tags (list of strings).
- Rule: The length of 'tokens' must equal the length of 'tags'.

Example:
[
  {{
    "sentence": "Credential zxcv987_12pqr-456tuv was verified in Tokyo at 192.168.1.1.",
    "tokens": ["Credential", "zxcv987_12pqr-456tuv", "was", "verified", "in", "Tokyo", "at", "192.168.1.1", "."],
    "tags": ["O", "B-Token", "O", "O", "O", "O", "O", "B-IP", "O"]
  }}
]

Generate {num_row} such rows.
"""
    )

    model = ChatOpenAI(model_name=model_name)
    chain = prompt | model
    response = chain.invoke({"num_row": num_row}).content

    parsed_data = clean_and_parse_llm_output(response)
    response = ast.literal_eval(parsed_data)

    rows = []
    for item in response:
        rows.append({
            "sentence": item["sentence"],
            "tokens": item["tokens"],
            "tags": item["tags"]
        })

    return pd.DataFrame(rows)
   

In [459]:
import re
import json
import pandas as pd
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI


def clean_and_parse_llm_output(text: str):
    # Step 1: Basic cleaning
    text = text.replace('```python', '').replace('```', '').strip()
    text = text.replace('“', '"').replace('”', '"').replace("‘", "'").replace("’", "'")

    # Remove non-printable characters except newline/tab
    text = ''.join(c for c in text if c.isprintable() or c in '\n\t')

    # Step 2: Convert tuples to JSON-style lists using regex
    text = re.sub(r'\((\d+),\s*(\d+),\s*"([^"]+)"\)', r'[\1, \2, "\3"]', text)
    
    return text

def generate_ner_ds_2(num_row: int, model_name: str) -> pd.DataFrame:
    prompt = PromptTemplate(
        input_variables=["num_row"],
        template="""
Generate a synthetic dataset for Named Entity Recognition (NER) in BIO format.
For each row:
- Provide a complex sentence containing at least one 'Token' entity, one 'IP' entity, and one or more other entities (e.g., geo, gpe, org, per, tim).
- Tokenize the sentence (split into tokens, ensuring proper handling of punctuation and special characters).
- Assign each token a BIO tag from the following options:
  'B-geo', 'B-gpe', 'B-org', 'B-per', 'B-tim', 'I-geo', 'I-gpe', 'I-org', 'I-per', 'I-tim', 'B-Token', 'B-IP', 'O'
  Where:
  - 'Token': A long, random string ( 20-100 characters) containing alphanumeric characters (letters and digits) and optionally the special characters '_' or '-'. The string must not resemble a name, date, or other meaningful entity. Example: '3tyslghiouyeuiosyet9w_euyi34-sfj'.
  - 'IP': A valid IPv4 (e.g., '192.168.1.101') or IPv6 (e.g., '2001:0db8:85a3:0000:0000:8a2e:0370:7334') address.
- Ensure that multi-word entities (e.g., 'New York') are tagged with 'B-' for the first token and 'I-' for subsequent tokens.
- Include at least one 'Token' and one 'IP' entity in each sentence, along with at least one other entity (geo, gpe, org, per, tim).

Output a list of dictionaries, where each dictionary contains:
- "sentence": the full sentence (string),
- "tokens": a list of tokens (list of strings),
- "tags": a list of corresponding BIO tags (list of strings).
- Rule: The length of 'tokens' must equal the length of 'tags'.

Example:
[
  {{
    "sentence": "At 14:30, 9x8w7v6u5t4s3r2q1p0o_lkjiuhygvfredcba authenticated the system at IP 8.8.8.8 in Berlin.",
    "tokens": ["At", "14:30", ",", "9x8w7v6u5t4s3r2q1p0o_lkjiuhygvfredcba", "authenticated", "the", "system", "at", "IP", "8.8.8.8", "in", "Berlin", "."],
    "tags": ["O", "B-tim", "O", "B-Token", "O", "O", "O", "O", "O", "B-IP", "O", "B-gpe", "O"]
  }},
  {{
    "sentence": "Facebook verified the credential abcd_efghijklmnopqrs4kj438-sdgrgy2_4hdsgdsk at 123.45.67.89 in London yesterday.",
    "tokens": ["Facebook", "verified", "the", "credential", "abcd_efghijklmnopqrs4kj438-sdgrgy2_4hdsgdsk", "at", "123.45.67.89", "in", "London", "yesterday", "."],
    "tags": ["B-org", "O", "O", "O", "B-Token", "O", "B-IP", "O", "B-gpe", "B-tim", "O"]
  }},
  {{
    "sentence": "zxcvbnmas4jlsh4_uy72dfghj-4klqwertyuiop was used on Tuesday from 10.10.10.10 in Tokyo.",
    "tokens": ["zxcvbnmas4jlsh4_uy72dfghj-4klqwertyuiop", "was", "used", "on", "Tuesday", "from", "10.10.10.10", "in", "Tokyo", "."],
    "tags": ["B-Token", "O", "O", "O", "B-tim", "O", "B-IP", "O", "B-geo", "O"]
  }},
  {{
    "sentence": "Dr. Jane Goodall accessed the system with code asdfghjklqwertyuiopzxcvbnm_2sdghkhhuy23jjyhs-3fe3efsg39hg at 203.0.113.76 on January 2025.",
    "tokens": ["Dr.", "Jane", "Goodall", "accessed", "the", "system", "with", "code", "asdfghjklqwertyuiopzxcvbnm_2sdghkhhuy23jjyhs-3fe3efsg39hg", "at", "203.0.113.76", "on", "January", "2025", "."],
    "tags": ["O", "B-per", "I-per", "O", "O", "O", "O", "O", "B-Token", "O", "B-IP", "O", "B-tim", "I-tim", "O"]
  }},
  {{
    "sentence": "The token system was updated in New York at 2001:0db8:0000:0000:0000:ff00:0042:8329 by qwer456_yuio789-123zxtsouisg23_werdsfoiu24uhv.",
    "tokens": ["The", "token", "system", "was", "updated", "in", "New", "York", "at", "2001:0db8:0000:0000:0000:ff00:0042:8329", "by", "qwer456_yuio789-123zxtsouisg23_werdsfoiu24uhv", "."],
    "tags": ["O", "O", "O", "O", "O", "O", "B-geo", "I-geo", "O", "B-IP", "O", "B-Token", "O"]
  }}
]


Generate {num_row} such rows.
"""
        
    )

    model = ChatOpenAI(model_name=model_name)
    chain = prompt | model
    response = chain.invoke({"num_row": num_row}).content
    
    parsed_data = clean_and_parse_llm_output(response)
    response = ast.literal_eval(parsed_data)
  
   
    # Extract into DataFrame
    rows = []
    for item in response:
        rows.append({
            "sentence": item["sentence"],
            "tokens": item["tokens"],
            "tags": item["tags"]
        })

    return pd.DataFrame(rows)


In [249]:
pd.set_option('display.max_colwidth', None)

In [520]:
dataset_generated_1 = generate_ner_test_1(num_row = 5,model_name = "gpt-4.1-mini")

SyntaxError: invalid syntax (<unknown>, line 2)

In [512]:
dataset_generated_1

,sentence,tokens,tags
0,The encrypted token 9q8w7e6r5t4y3u2i1o0p_lkjhgfdsaz was accessed from 10.0.0.254 during the audit.,"[The, encrypted, token, 9q8w7e6r5t4y3u2i1o0p_lkjhgfdsaz, was, accessed, from, 10.0.0.254, during, the, audit, .]","[O, O, O, B-Token, O, O, O, B-IP, O, O, O, O]"
1,192.168.100.23 received data from the key x1y2z3a4b5c6d7e8f9_g0h1j2k3l4m5n6op.,"[192.168.100.23, received, data, from, the, key, x1y2z3a4b5c6d7e8f9_g0h1j2k3l4m5n6op, .]","[B-IP, O, O, O, O, O, B-Token, O]"
2,A temporary identifier qwertyuiopasdfghjklzxcvbnm-1234567 was verified alongside 172.31.255.255.,"[A, temporary, identifier, qwertyuiopasdfghjklzxcvbnm-1234567, was, verified, alongside, 172.31.255.255, .]","[O, O, O, B-Token, O, O, O, B-IP, O]"
3,Users sent requests using the token mnbvcxzlkjhgfdsapoiuytrewq-0987654321 from 2001:0db8:85a3:0000:0000:8a2e:0370:7334.,"[Users, sent, requests, using, the, token, mnbvcxzlkjhgfdsapoiuytrewq-0987654321, from, 2001:0db8:85a3:0000:0000:8a2e:0370:7334, .]","[O, O, O, O, O, O, B-Token, O, B-IP, O]"
4,"During the backup, the token 1a2b3c4d5e6f7g8h9i0jkl_mn-opqrs was stored at 192.0.2.146 securely.","[During, the, backup, ,, the, token, 1a2b3c4d5e6f7g8h9i0jkl_mn-opqrs, was, stored, at, 192.0.2.146, securely, .]","[O, O, O, O, O, O, B-Token, O, O, O, B-IP, O, O]"
5,The server logged the activity of 2607:f0d0:1002:51::4 and token abcdefghijklmnopqr_stuvwx-yz1234567890.,"[The, server, logged, the, activity, of, 2607:f0d0:1002:51::4, and, token, abcdefghijklmnopqr_stuvwx-yz1234567890, .]","[O, O, O, O, O, O, B-IP, O, O, B-Token, O]"
6,An alert was triggered by 198.51.100.77 after receiving token zxcvbnm_1234567890poiuytrewq-lkjhgf.,"[An, alert, was, triggered, by, 198.51.100.77, after, receiving, token, zxcvbnm_1234567890poiuytrewq-lkjhgf, .]","[O, O, O, O, O, B-IP, O, O, O, B-Token, O]"
7,The IP address 203.0.113.195 communicated with the token lkjhgfdsapoiuytrewq_mnbvcxz-0987654321.,"[The, IP, address, 203.0.113.195, communicated, with, the, token, lkjhgfdsapoiuytrewq_mnbvcxz-0987654321, .]","[O, O, O, B-IP, O, O, O, O, B-Token, O]"
8,"After verification, token 9q8w7e6r5t4y3u2i1o0p_lkjhgfdsaz was sent from fe80::1ff:fe23:4567:890a.","[After, verification, ,, token, 9q8w7e6r5t4y3u2i1o0p_lkjhgfdsaz, was, sent, from, fe80::1ff:fe23:4567:890a, .]","[O, O, O, O, B-Token, O, O, O, B-IP, O]"


In [456]:
dataset_generated_1

,sentence,tokens,tags
0,Microsoft deployed the security patch 4j5k6l7m8n9opqr_stuvwxy-1234567890abcdef at 172.16.254.1 during the meeting in San Francisco yesterday.,"[Microsoft, deployed, the, security, patch, 4j5k6l7m8n9opqr_stuvwxy-1234567890abcdef, at, 172.16.254.1, during, the, meeting, in, San, Francisco, yesterday, .]","[B-org, O, O, O, O, B-Token, O, B-IP, O, O, O, O, B-geo, I-geo, B-tim, O]"
1,"At 09:15, Dr. Helena Schmidt sent the encrypted message 9q8w7e6r5t4y3u2i1o0p_lkjhgfdsa-zxvbnm from IP 10.0.0.254 in Berlin.","[At, 09:15, ,, Dr., Helena, Schmidt, sent, the, encrypted, message, 9q8w7e6r5t4y3u2i1o0p_lkjhgfdsa-zxvbnm, from, IP, 10.0.0.254, in, Berlin, .]","[O, B-tim, O, O, B-per, I-per, O, O, O, O, B-Token, O, O, B-IP, O, B-gpe, O]"
2,The conference in New Delhi started at 2023-11-25T08:00 with session code zx12-34_abcd567890efghijklmno at 192.0.2.146.,"[The, conference, in, New, Delhi, started, at, 2023-11-25T08:00, with, session, code, zx12-34_abcd567890efghijklmno, at, 192.0.2.146, .]","[O, O, O, B-geo, I-geo, O, O, B-tim, O, O, O, B-Token, O, B-IP, O]"
3,Jessica and Michael sent the file qw987asdfghjklzxcvbnm_12345-67890 to 2001:db8::ff00:42:8329 before noon.,"[Jessica, and, Michael, sent, the, file, qw987asdfghjklzxcvbnm_12345-67890, to, 2001:db8::ff00:42:8329, before, noon, .]","[B-per, O, B-per, O, O, O, B-Token, O, B-IP, O, B-tim, O]"
4,Google updated their logs with token r4nd0m_5tr1ng-abcdef1234567890 at 8.8.4.4 during the event in Toronto last week.,"[Google, updated, their, logs, with, token, r4nd0m_5tr1ng-abcdef1234567890, at, 8.8.4.4, during, the, event, in, Toronto, last, week, .]","[B-org, O, O, O, O, O, B-Token, O, B-IP, O, O, O, O, B-geo, B-tim, I-tim, O]"
5,"On Monday, the access code 1234567890abcdef-ghijklmnopqrstuvwx was registered from IP 255.255.255.0 in Los Angeles.","[On, Monday, ,, the, access, code, 1234567890abcdef-ghijklmnopqrstuvwx, was, registered, from, IP, 255.255.255.0, in, Los, Angeles, .]","[O, B-tim, O, O, O, O, B-Token, O, O, O, O, B-IP, O, B-geo, I-geo, O]"
6,The user named John O'Connor generated key asdfghjklqwertyuiopzxcvbnm_0987654321 at 172.31.255.255 in Dublin this morning.,"[The, user, named, John, O'Connor, generated, key, asdfghjklqwertyuiopzxcvbnm_0987654321, at, 172.31.255.255, in, Dublin, this, morning, .]","[O, O, O, B-per, I-per, O, O, B-Token, O, B-IP, O, B-geo, O, B-tim, O]"
7,Amazon reported abnormal activity with token zxcvbnmlkjhgfdsa0987654321qwertyuiop-123 at 203.0.113.195 near Seattle yesterday afternoon.,"[Amazon, reported, abnormal, activity, with, token, zxcvbnmlkjhgfdsa0987654321qwertyuiop-123, at, 203.0.113.195, near, Seattle, yesterday, afternoon, .]","[B-org, O, O, O, O, O, B-Token, O, B-IP, O, B-geo, B-tim, I-tim, O]"
8,Prof. Alan Turing uploaded the file qwerty-12345_asdfghjkl_zxcvbnm9876543210 at 2001:0db8:85a3::8a2e:0370:7334 during the afternoon in Cambridge.,"[Prof., Alan, Turing, uploaded, the, file, qwerty-12345_asdfghjkl_zxcvbnm9876543210, at, 2001:0db8:85a3::8a2e:0370:7334, during, the, afternoon, in, Cambridge, .]","[O, B-per, I-per, O, O, O, B-Token, O, B-IP, O, O, B-tim, O, B-geo, O]"
9,The update code 5f6e7d8c9b0a1fghijklmno_pqrstuvwx-yz1234567890 was sent from 198.51.100.24 to the headquarters in Washington D.C. last Friday.,"[The, update, code, 5f6e7d8c9b0a1fghijklmno_pqrstuvwx-yz1234567890, was, sent, from, 198.51.100.24, to, the, headquarters, in, Washington, D.C., last, Friday, .]","[O, O, O, B-Token, O, O, O, B-IP, O, O, O, O, B-geo, I-geo, B-tim, I-tim, O]"


In [457]:
dataset_generated_1["Indicator"] = dataset_generated_1.apply(
    lambda row: len(row["tags"]) == len(row["tokens"]), axis=1
)

In [458]:
dataset_generated_1

,sentence,tokens,tags,Indicator
0,Microsoft deployed the security patch 4j5k6l7m8n9opqr_stuvwxy-1234567890abcdef at 172.16.254.1 during the meeting in San Francisco yesterday.,"[Microsoft, deployed, the, security, patch, 4j5k6l7m8n9opqr_stuvwxy-1234567890abcdef, at, 172.16.254.1, during, the, meeting, in, San, Francisco, yesterday, .]","[B-org, O, O, O, O, B-Token, O, B-IP, O, O, O, O, B-geo, I-geo, B-tim, O]",True
1,"At 09:15, Dr. Helena Schmidt sent the encrypted message 9q8w7e6r5t4y3u2i1o0p_lkjhgfdsa-zxvbnm from IP 10.0.0.254 in Berlin.","[At, 09:15, ,, Dr., Helena, Schmidt, sent, the, encrypted, message, 9q8w7e6r5t4y3u2i1o0p_lkjhgfdsa-zxvbnm, from, IP, 10.0.0.254, in, Berlin, .]","[O, B-tim, O, O, B-per, I-per, O, O, O, O, B-Token, O, O, B-IP, O, B-gpe, O]",True
2,The conference in New Delhi started at 2023-11-25T08:00 with session code zx12-34_abcd567890efghijklmno at 192.0.2.146.,"[The, conference, in, New, Delhi, started, at, 2023-11-25T08:00, with, session, code, zx12-34_abcd567890efghijklmno, at, 192.0.2.146, .]","[O, O, O, B-geo, I-geo, O, O, B-tim, O, O, O, B-Token, O, B-IP, O]",True
3,Jessica and Michael sent the file qw987asdfghjklzxcvbnm_12345-67890 to 2001:db8::ff00:42:8329 before noon.,"[Jessica, and, Michael, sent, the, file, qw987asdfghjklzxcvbnm_12345-67890, to, 2001:db8::ff00:42:8329, before, noon, .]","[B-per, O, B-per, O, O, O, B-Token, O, B-IP, O, B-tim, O]",True
4,Google updated their logs with token r4nd0m_5tr1ng-abcdef1234567890 at 8.8.4.4 during the event in Toronto last week.,"[Google, updated, their, logs, with, token, r4nd0m_5tr1ng-abcdef1234567890, at, 8.8.4.4, during, the, event, in, Toronto, last, week, .]","[B-org, O, O, O, O, O, B-Token, O, B-IP, O, O, O, O, B-geo, B-tim, I-tim, O]",True
5,"On Monday, the access code 1234567890abcdef-ghijklmnopqrstuvwx was registered from IP 255.255.255.0 in Los Angeles.","[On, Monday, ,, the, access, code, 1234567890abcdef-ghijklmnopqrstuvwx, was, registered, from, IP, 255.255.255.0, in, Los, Angeles, .]","[O, B-tim, O, O, O, O, B-Token, O, O, O, O, B-IP, O, B-geo, I-geo, O]",True
6,The user named John O'Connor generated key asdfghjklqwertyuiopzxcvbnm_0987654321 at 172.31.255.255 in Dublin this morning.,"[The, user, named, John, O'Connor, generated, key, asdfghjklqwertyuiopzxcvbnm_0987654321, at, 172.31.255.255, in, Dublin, this, morning, .]","[O, O, O, B-per, I-per, O, O, B-Token, O, B-IP, O, B-geo, O, B-tim, O]",True
7,Amazon reported abnormal activity with token zxcvbnmlkjhgfdsa0987654321qwertyuiop-123 at 203.0.113.195 near Seattle yesterday afternoon.,"[Amazon, reported, abnormal, activity, with, token, zxcvbnmlkjhgfdsa0987654321qwertyuiop-123, at, 203.0.113.195, near, Seattle, yesterday, afternoon, .]","[B-org, O, O, O, O, O, B-Token, O, B-IP, O, B-geo, B-tim, I-tim, O]",True
8,Prof. Alan Turing uploaded the file qwerty-12345_asdfghjkl_zxcvbnm9876543210 at 2001:0db8:85a3::8a2e:0370:7334 during the afternoon in Cambridge.,"[Prof., Alan, Turing, uploaded, the, file, qwerty-12345_asdfghjkl_zxcvbnm9876543210, at, 2001:0db8:85a3::8a2e:0370:7334, during, the, afternoon, in, Cambridge, .]","[O, B-per, I-per, O, O, O, B-Token, O, B-IP, O, O, B-tim, O, B-geo, O]",True
9,The update code 5f6e7d8c9b0a1fghijklmno_pqrstuvwx-yz1234567890 was sent from 198.51.100.24 to the headquarters in Washington D.C. last Friday.,"[The, update, code, 5f6e7d8c9b0a1fghijklmno_pqrstuvwx-yz1234567890, was, sent, from, 198.51.100.24, to, the, headquarters, in, Washington, D.C., last, Friday, .]","[O, O, O, B-Token, O, O, O, B-IP, O, O, O, O, B-geo, I-geo, B-tim, I-tim, O]",True


In [460]:
dataset_generated_2 = generate_ner_ds_2(num_row = 50,model_name = "gpt-4.1-mini")

In [482]:
dataset_generated_2

,sentence,tokens,tags,Indicator
0,"Yesterday at 18:45, the engineer 1qaz2wsx3edc4rfv5tgb6yhn7ujm8iklop_123 accessed the server from IP 172.16.254.1 in San Francisco.","[Yesterday, at, 18:45, ,, the, engineer, 1qaz2wsx3edc4rfv5tgb6yhn7ujm8iklop_123, accessed, the, server, from, IP, 172.16.254.1, in, San, Francisco, .]","[B-tim, O, B-tim, O, O, O, B-Token, O, O, O, O, O, B-IP, O, B-geo, I-geo, O]",True
1,"At 09:00 AM, CEO Alice Johnson uploaded the file zxvbnm_09876lkjhgfdsapoiu-qwer123 to IP 10.0.0.254 from New York headquarters.","[At, 09:00, AM, ,, CEO, Alice, Johnson, uploaded, the, file, zxvbnm_09876lkjhgfdsapoiu-qwer123, to, IP, 10.0.0.254, from, New, York, headquarters, .]","[O, B-tim, I-tim, O, O, B-per, I-per, O, O, O, B-Token, O, O, B-IP, O, B-geo, I-geo, O, O]",True
2,The device x1y2z3a4b5c6d7e8f9g0_hjklmno-pqrstuv was pinged at IP 192.0.2.146 during the Berlin conference.,"[The, device, x1y2z3a4b5c6d7e8f9g0_hjklmno-pqrstuv, was, pinged, at, IP, 192.0.2.146, during, the, Berlin, conference, .]","[O, O, B-Token, O, O, O, O, B-IP, O, O, B-geo, O, O]",True
3,Google's server 9ujm8iklop1234567890_qwertyuiop-asdfghjkl was attacked from IP 203.0.113.195 in Tokyo at dawn.,"[Google, 's, server, 9ujm8iklop1234567890_qwertyuiop-asdfghjkl, was, attacked, from, IP, 203.0.113.195, in, Tokyo, at, dawn, .]","[B-org, O, O, B-Token, O, O, O, O, B-IP, O, B-geo, O, B-tim, O]",True
4,Dr. Henry Wu used the token asdfghjklqwertyuiopzxcvbnm_12345 during the IP scan at 2001:db8::1 in Paris.,"[Dr., Henry, Wu, used, the, token, asdfghjklqwertyuiopzxcvbnm_12345, during, the, IP, scan, at, 2001:db8::1, in, Paris, .]","[O, B-per, I-per, O, O, O, B-Token, O, O, O, O, O, B-IP, O, B-geo, O]",True
5,"At the Microsoft conference, the token zxcvbnmasdfghjklqwertyuiop_0987654321 was assigned to IP 198.51.100.23 in Seattle.","[At, the, Microsoft, conference, ,, the, token, zxcvbnmasdfghjklqwertyuiop_0987654321, was, assigned, to, IP, 198.51.100.23, in, Seattle, .]","[O, O, B-org, O, O, O, O, B-Token, O, O, O, O, B-IP, O, B-geo, O]",True
6,"On Monday, the analyst 5tgb6yhn7ujm8iklop_1qaz2wsx3edc investigated IP 172.217.10.14 in Mountain View.","[On, Monday, ,, the, analyst, 5tgb6yhn7ujm8iklop_1qaz2wsx3edc, investigated, IP, 172.217.10.14, in, Mountain, View, .]","[O, B-tim, O, O, O, B-Token, O, O, B-IP, O, B-geo, I-geo, O]",True
7,Amazon's token qwertyuiopasdfghjklzxcvbnm_1234 was logged from IP 54.239.25.192 in Dublin yesterday.,"[Amazon, 's, token, qwertyuiopasdfghjklzxcvbnm_1234, was, logged, from, IP, 54.239.25.192, in, Dublin, yesterday, .]","[B-org, O, O, B-Token, O, O, O, O, B-IP, O, B-geo, B-tim, O]",True
8,The investigator Jane_Doe1234567890-token checked IP 2607:f8b0:4005:805::200e during the London summit.,"[The, investigator, Jane_Doe1234567890-token, checked, IP, 2607:f8b0:4005:805::200e, during, the, London, summit, .]","[O, O, B-Token, O, O, B-IP, O, O, B-geo, O, O]",True
9,"At 03:15 PM, the token 7ujm8iklopqazwsxedcrfvtgbnhy_09876 was used to access IP 131.107.0.89 in Chicago.","[At, 03:15, PM, ,, the, token, 7ujm8iklopqazwsxedcrfvtgbnhy_09876, was, used, to, access, IP, 131.107.0.89, in, Chicago, .]","[O, B-tim, I-tim, O, O, O, B-Token, O, O, O, O, O, B-IP, O, B-geo, O]",True


In [462]:
dataset_generated_2["Indicator"] = dataset_generated_2.apply(
    lambda row: len(row["tags"]) == len(row["tokens"]), axis=1
)

In [ ]:
dataset_generated_2

In [463]:
dataset_generated_3 = generate_ner_ds_2(num_row = 50,model_name = "gpt-4.1-mini")

In [483]:
dataset_generated_3

,sentence,tokens,tags
0,Amazon's server reported the token 9ab8c7d6e5f4g3h2i1j0klmn-opqrstu at 10.0.0.45 during the meeting in Berlin.,"[Amazon, 's, server, reported, the, token, 9ab8c7d6e5f4g3h2i1j0klmn-opqrstu, at, 10.0.0.45, during, the, meeting, in, Berlin, .]","[B-org, O, O, O, O, O, B-Token, O, B-IP, O, O, O, O, B-geo, O]"
1,Jessica sent credential x7v6w5u4t3s2r1q0p9o8n7mlk-jihgfedcba from 172.217.3.110 before noon in San Francisco.,"[Jessica, sent, credential, x7v6w5u4t3s2r1q0p9o8n7mlk-jihgfedcba, from, 172.217.3.110, before, noon, in, San, Francisco, .]","[B-per, O, O, B-Token, O, B-IP, O, B-tim, O, B-geo, I-geo, O]"
2,"On Tuesday, the user accessed system with key uvwxyz12345abcdef-ghijklmno from 2607:f8b0:4005:805::200e.","[On, Tuesday, ,, the, user, accessed, system, with, key, uvwxyz12345abcdef-ghijklmno, from, 2607:f8b0:4005:805::200e, .]","[O, B-tim, O, O, O, O, O, O, O, B-Token, O, B-IP, O]"
3,The IP address 192.0.2.146 was used by Elon Musk to verify the code qwertyuiopasdfghjklzxcvbnm1234.,"[The, IP, address, 192.0.2.146, was, used, by, Elon, Musk, to, verify, the, code, qwertyuiopasdfghjklzxcvbnm1234, .]","[O, O, O, B-IP, O, O, O, B-per, I-per, O, O, O, O, B-Token, O]"
4,Credential 1a2b3c4d5e6f7g8h9i0jklmnopqrstu was authenticated at the headquarters of Microsoft in Seattle.,"[Credential, 1a2b3c4d5e6f7g8h9i0jklmnopqrstu, was, authenticated, at, the, headquarters, of, Microsoft, in, Seattle, .]","[O, B-Token, O, O, O, O, O, O, B-org, O, B-gpe, O]"
5,"Yesterday, the token z1x2c3v4b5n6m7a8s9d0fqwer-tyuio was used from 203.0.113.76 in New York City.","[Yesterday, ,, the, token, z1x2c3v4b5n6m7a8s9d0fqwer-tyuio, was, used, from, 203.0.113.76, in, New, York, City, .]","[B-tim, O, O, O, B-Token, O, O, O, B-IP, O, B-geo, I-geo, I-geo, O]"
6,Google's security team verified the random string 4r5e6t7y8u9i0opasdfghjkl-zxcvbnm from IP 198.51.100.23 yesterday.,"[Google, 's, security, team, verified, the, random, string, 4r5e6t7y8u9i0opasdfghjkl-zxcvbnm, from, IP, 198.51.100.23, yesterday, .]","[B-org, O, O, O, O, O, O, O, B-Token, O, O, B-IP, B-tim, O]"
7,Michael Johnson accessed the platform with code asdfghjklqwertyuiopzxcvbnm1234567890 at 172.31.255.255 on Friday.,"[Michael, Johnson, accessed, the, platform, with, code, asdfghjklqwertyuiopzxcvbnm1234567890, at, 172.31.255.255, on, Friday, .]","[B-per, I-per, O, O, O, O, O, B-Token, O, B-IP, O, B-tim, O]"
8,The server located in Paris received the key qwertyuiopasdfghjklzxcvbnm0987654321 from 2400:cb00:2048:1::c629:d7a2.,"[The, server, located, in, Paris, received, the, key, qwertyuiopasdfghjklzxcvbnm0987654321, from, 2400:cb00:2048:1::c629:d7a2, .]","[O, O, O, O, B-geo, O, O, O, B-Token, O, B-IP, O]"
9,"On Sunday, the organization OpenAI verified token 1234567890abcdef-ghijklmnopqrstu from IP 192.168.0.101.","[On, Sunday, ,, the, organization, OpenAI, verified, token, 1234567890abcdef-ghijklmnopqrstu, from, IP, 192.168.0.101, .]","[O, B-tim, O, O, O, B-org, O, O, B-Token, O, O, B-IP, O]"


In [484]:
dataset_generated_3["Indicator"] = dataset_generated_3.apply(
    lambda row: len(row["tags"]) == len(row["tokens"]), axis=1
)

In [465]:
dataset_generated_4 = generate_ner_ds_2(num_row = 50,model_name = "gpt-4.1-mini")

In [485]:
dataset_generated_4

,sentence,tokens,tags,Indicator
0,Amazon's server received the token 9xw8v7u6t5r4q3p2_lkji0 from IP 203.0.113.195 during the event in Seattle.,"[Amazon, 's, server, received, the, token, 9xw8v7u6t5r4q3p2_lkji0, from, IP, 203.0.113.195, during, the, event, in, Seattle, .]","[B-org, O, O, O, O, O, B-Token, O, O, B-IP, O, O, O, O, B-geo, O]",True
1,"In Berlin, the credential fghj7890_klmn-12345 was used to access 10.10.10.10 at midnight.","[In, Berlin, ,, the, credential, fghj7890_klmn-12345, was, used, to, access, 10.10.10.10, at, midnight, .]","[O, B-geo, O, O, O, B-Token, O, O, O, O, B-IP, O, B-tim, O]",True
2,The token p0o9i8u7y6t5r4e3-wqazx was sent from 172.20.10.5 to the headquarters of Microsoft in Redmond.,"[The, token, p0o9i8u7y6t5r4e3-wqazx, was, sent, from, 172.20.10.5, to, the, headquarters, of, Microsoft, in, Redmond, .]","[O, O, B-Token, O, O, O, B-IP, O, O, O, O, B-org, O, B-geo, O]",True
3,"Yesterday, CEO Satya Nadella logged in using the code asdfqwer_12345-67890 from 2607:f8b0:4005:805::200e.","[Yesterday, ,, CEO, Satya, Nadella, logged, in, using, the, code, asdfqwer_12345-67890, from, 2607:f8b0:4005:805::200e, .]","[B-tim, O, O, B-per, I-per, O, O, O, O, O, B-Token, O, B-IP, O]",True
4,"During the conference in New York, the encrypted token zxcvbnm_09876-asdfgh was validated with IP 198.51.100.22.","[During, the, conference, in, New, York, ,, the, encrypted, token, zxcvbnm_09876-asdfgh, was, validated, with, IP, 198.51.100.22, .]","[O, O, O, O, B-geo, I-geo, O, O, O, O, B-Token, O, O, O, O, B-IP, O]",True
5,"At 14:30, the token nm9k8j7h6g5f4d3s2-aqwer was received from 192.0.2.44 during the summit in Paris.","[At, 14:30, ,, the, token, nm9k8j7h6g5f4d3s2-aqwer, was, received, from, 192.0.2.44, during, the, summit, in, Paris, .]","[O, B-tim, O, O, O, B-Token, O, O, O, B-IP, O, O, O, O, B-geo, O]",True
6,Google's engineer used a key 7yu8i9o0p-asdfghjk_lmn12 to connect from 172.31.255.255 in California.,"[Google, 's, engineer, used, a, key, 7yu8i9o0p-asdfghjk_lmn12, to, connect, from, 172.31.255.255, in, California, .]","[B-org, O, O, O, O, O, B-Token, O, O, O, B-IP, O, B-geo, O]",True
7,Angela Merkel accessed the system using token qwer1234_asdf5678-zxcvbn from IP 10.0.0.1 on Tuesday.,"[Angela, Merkel, accessed, the, system, using, token, qwer1234_asdf5678-zxcvbn, from, IP, 10.0.0.1, on, Tuesday, .]","[B-per, I-per, O, O, O, O, O, B-Token, O, O, B-IP, O, B-tim, O]",True
8,The encrypted credential asdfghjklqwertyuiop_1234567890 was verified at 8.8.8.8 during the meeting in London.,"[The, encrypted, credential, asdfghjklqwertyuiop_1234567890, was, verified, at, 8.8.8.8, during, the, meeting, in, London, .]","[O, O, O, B-Token, O, O, O, B-IP, O, O, O, O, B-geo, O]",True
9,Microsoft's server logged the code 1qaz2wsx3edc4rfv5tgb from IPv6 address fe80::1ff:fe23:4567:890a on Wednesday.,"[Microsoft, 's, server, logged, the, code, 1qaz2wsx3edc4rfv5tgb, from, IPv6, address, fe80::1ff:fe23:4567:890a, on, Wednesday, .]","[B-org, O, O, O, O, O, B-Token, O, O, O, B-IP, O, B-tim, O]",True


In [486]:
dataset_generated_4["Indicator"] = dataset_generated_4.apply(
    lambda row: len(row["tags"]) == len(row["tokens"]), axis=1
)

In [487]:
dataset_generated_4

,sentence,tokens,tags,Indicator
0,Amazon's server received the token 9xw8v7u6t5r4q3p2_lkji0 from IP 203.0.113.195 during the event in Seattle.,"[Amazon, 's, server, received, the, token, 9xw8v7u6t5r4q3p2_lkji0, from, IP, 203.0.113.195, during, the, event, in, Seattle, .]","[B-org, O, O, O, O, O, B-Token, O, O, B-IP, O, O, O, O, B-geo, O]",True
1,"In Berlin, the credential fghj7890_klmn-12345 was used to access 10.10.10.10 at midnight.","[In, Berlin, ,, the, credential, fghj7890_klmn-12345, was, used, to, access, 10.10.10.10, at, midnight, .]","[O, B-geo, O, O, O, B-Token, O, O, O, O, B-IP, O, B-tim, O]",True
2,The token p0o9i8u7y6t5r4e3-wqazx was sent from 172.20.10.5 to the headquarters of Microsoft in Redmond.,"[The, token, p0o9i8u7y6t5r4e3-wqazx, was, sent, from, 172.20.10.5, to, the, headquarters, of, Microsoft, in, Redmond, .]","[O, O, B-Token, O, O, O, B-IP, O, O, O, O, B-org, O, B-geo, O]",True
3,"Yesterday, CEO Satya Nadella logged in using the code asdfqwer_12345-67890 from 2607:f8b0:4005:805::200e.","[Yesterday, ,, CEO, Satya, Nadella, logged, in, using, the, code, asdfqwer_12345-67890, from, 2607:f8b0:4005:805::200e, .]","[B-tim, O, O, B-per, I-per, O, O, O, O, O, B-Token, O, B-IP, O]",True
4,"During the conference in New York, the encrypted token zxcvbnm_09876-asdfgh was validated with IP 198.51.100.22.","[During, the, conference, in, New, York, ,, the, encrypted, token, zxcvbnm_09876-asdfgh, was, validated, with, IP, 198.51.100.22, .]","[O, O, O, O, B-geo, I-geo, O, O, O, O, B-Token, O, O, O, O, B-IP, O]",True
5,"At 14:30, the token nm9k8j7h6g5f4d3s2-aqwer was received from 192.0.2.44 during the summit in Paris.","[At, 14:30, ,, the, token, nm9k8j7h6g5f4d3s2-aqwer, was, received, from, 192.0.2.44, during, the, summit, in, Paris, .]","[O, B-tim, O, O, O, B-Token, O, O, O, B-IP, O, O, O, O, B-geo, O]",True
6,Google's engineer used a key 7yu8i9o0p-asdfghjk_lmn12 to connect from 172.31.255.255 in California.,"[Google, 's, engineer, used, a, key, 7yu8i9o0p-asdfghjk_lmn12, to, connect, from, 172.31.255.255, in, California, .]","[B-org, O, O, O, O, O, B-Token, O, O, O, B-IP, O, B-geo, O]",True
7,Angela Merkel accessed the system using token qwer1234_asdf5678-zxcvbn from IP 10.0.0.1 on Tuesday.,"[Angela, Merkel, accessed, the, system, using, token, qwer1234_asdf5678-zxcvbn, from, IP, 10.0.0.1, on, Tuesday, .]","[B-per, I-per, O, O, O, O, O, B-Token, O, O, B-IP, O, B-tim, O]",True
8,The encrypted credential asdfghjklqwertyuiop_1234567890 was verified at 8.8.8.8 during the meeting in London.,"[The, encrypted, credential, asdfghjklqwertyuiop_1234567890, was, verified, at, 8.8.8.8, during, the, meeting, in, London, .]","[O, O, O, B-Token, O, O, O, B-IP, O, O, O, O, B-geo, O]",True
9,Microsoft's server logged the code 1qaz2wsx3edc4rfv5tgb from IPv6 address fe80::1ff:fe23:4567:890a on Wednesday.,"[Microsoft, 's, server, logged, the, code, 1qaz2wsx3edc4rfv5tgb, from, IPv6, address, fe80::1ff:fe23:4567:890a, on, Wednesday, .]","[B-org, O, O, O, O, O, B-Token, O, O, O, B-IP, O, B-tim, O]",True


In [468]:
dataset_generated_5 = generate_ner_ds_2(num_row = 50,model_name = "gpt-4.1-mini")

In [470]:
dataset_generated_5["Indicator"] = dataset_generated_5.apply(
    lambda row: len(row["tags"]) == len(row["tokens"]), axis=1
)

In [488]:
dataset_generated_5

,sentence,tokens,tags,Indicator
0,Sarah Connor sent the code a1b2c3d4e5f6g7h8i9j0klmnqrs from 10.0.0.42 to Microsoft headquarters in Redmond.,"[Sarah, Connor, sent, the, code, a1b2c3d4e5f6g7h8i9j0klmnqrs, from, 10.0.0.42, to, Microsoft, headquarters, in, Redmond, .]","[B-per, I-per, O, O, O, B-Token, O, B-IP, O, B-org, O, O, B-gpe, O]",True
1,The token zxy_987-lkji654321 was authenticated at 172.16.254.1 during the Berlin conference.,"[The, token, zxy_987-lkji654321, was, authenticated, at, 172.16.254.1, during, the, Berlin, conference, .]","[O, O, B-Token, O, O, O, B-IP, O, O, B-geo, O, O]",True
2,Google verified the credential qwerty_12345-asdfgh67890 from 2001:0db8:85a3:0000:0000:8a2e:0370:7334 last Friday.,"[Google, verified, the, credential, qwerty_12345-asdfgh67890, from, 2001:0db8:85a3:0000:0000:8a2e:0370:7334, last, Friday, .]","[B-org, O, O, O, B-Token, O, B-IP, O, B-tim, O]",True
3,"At 192.168.0.99, the key asdfghjklqwertyuiop1234567890 was used by Alice in Paris.","[At, 192.168.0.99, ,, the, key, asdfghjklqwertyuiop1234567890, was, used, by, Alice, in, Paris, .]","[O, B-IP, O, O, O, B-Token, O, O, O, B-per, O, B-geo, O]",True
4,"During the meeting in New York, the credential mnopqrstuvwxyz12345 was sent from 8.8.8.8.","[During, the, meeting, in, New, York, ,, the, credential, mnopqrstuvwxyzz12345, was, sent, from, 8.8.8.8, .]","[O, O, O, O, B-geo, I-geo, O, O, O, B-Token, O, O, O, B-IP, O]",True
5,Amazon's token abcd_efgh-ijklmnopqr was verified near 172.217.3.110 at noon.,"[Amazon, 's, token, abcd_efgh-ijklmnopqr, was, verified, near, 172.217.3.110, at, noon, .]","[B-org, O, O, B-Token, O, O, O, B-IP, O, B-tim, O]",True
6,The token 1a2b3c4d5e6f7g8h9i0jklmnop was accessed by Michael Johnson from 2400:cb00:2048:1::c629:d7a2.,"[The, token, 1a2b3c4d5e6f7g8h9i0jklmnop, was, accessed, by, Michael, Johnson, from, 2400:cb00:2048:1::c629:d7a2, .]","[O, O, B-Token, O, O, O, B-per, I-per, O, B-IP, O]",True
7,"In London, the key uvwxyz_12345-abcd67890efgh was sent from 203.0.113.45 yesterday.","[In, London, ,, the, key, uvwxyz_12345-abcd67890efgh, was, sent, from, 203.0.113.45, yesterday, .]","[O, B-geo, O, O, O, B-Token, O, O, O, B-IP, B-tim, O]",True
8,Facebook authenticated credential zxcvbnm_09876-asdfghjklqwerty from 198.51.100.23 during the summit.,"[Facebook, authenticated, credential, zxcvbnm_09876-asdfghjklqwerty, from, 198.51.100.23, during, the, summit, .]","[B-org, O, O, B-Token, O, B-IP, O, O, O, O]",True
9,The conference at 2607:f8b0:4005:805::200e started with token qwertyuiopasdfghjklzxcvbnm123456.,"[The, conference, at, 2607:f8b0:4005:805::200e, started, with, token, qwertyuiopasdfghjklzxcvbnm123456, .]","[O, O, O, B-IP, O, O, O, B-Token, O]",True


In [472]:
dataset_generated_6 = generate_ner_ds_2(num_row = 50,model_name = "gpt-4.1-mini")

In [489]:
dataset_generated_6

,sentence,tokens,tags,Indicator
0,Alice sent the token ab12_xy34-pqrs7890 from 10.0.0.42 to Berlin headquarters.,"[Alice, sent, the, token, ab12_xy34-pqrs7890, from, 10.0.0.42, to, Berlin, headquarters, .]","[B-per, O, O, O, B-Token, O, B-IP, O, B-gpe, O, O]",True
1,"On Wednesday, the server at 172.16.254.1 processed credential 9w8e7_r6t5y4-u3i2o1p for Amazon.","[On, Wednesday, ,, the, server, at, 172.16.254.1, processed, credential, 9w8e7_r6t5y4-u3i2o1p, for, Amazon, .]","[O, B-tim, O, O, O, O, B-IP, O, O, B-Token, O, B-org, O]",True
2,The key zx7vbnm_0987-asdfghjk was verified during the meeting in New York with John Doe.,"[The, key, zx7vbnm_0987-asdfghjk, was, verified, during, the, meeting, in, New, York, with, John, Doe, .]","[O, O, B-Token, O, O, O, O, O, O, B-gpe, I-gpe, O, B-per, I-per, O]",True
3,IBM authenticated device 3tyslghiouyeuiosyet9w_euyi34-sfj connected from 2001:0db8:85a3:0000:0000:8a2e:0370:7334.,"[IBM, authenticated, device, 3tyslghiouyeuiosyet9w_euyi34-sfj, connected, from, 2001:0db8:85a3:0000:0000:8a2e:0370:7334, .]","[B-org, O, O, B-Token, O, O, B-IP, O]",True
4,"Yesterday, the token fgh56_jklm78-xyz12345 was sent to Paris from 192.0.2.146.","[Yesterday, ,, the, token, fgh56_jklm78-xyz12345, was, sent, to, Paris, from, 192.0.2.146, .]","[B-tim, O, O, O, B-Token, O, O, O, B-geo, O, B-IP, O]",True
5,The CEO Michael Smith accessed the system using code qwer1234_asdf5678-ghjkl on 2023-04-01 from 203.0.113.195.,"[The, CEO, Michael, Smith, accessed, the, system, using, code, qwer1234_asdf5678-ghjkl, on, 2023-04-01, from, 203.0.113.195, .]","[O, O, B-per, I-per, O, O, O, O, O, B-Token, O, O, O, B-IP, O]",True
6,Token rmnbv54321_qwer-tyuio was generated at 198.51.100.23 during the conference in London.,"[Token, rmnbv54321_qwer-tyuio, was, generated, at, 198.51.100.23, during, the, conference, in, London, .]","[O, B-Token, O, O, O, B-IP, O, O, O, O, B-gpe, O]",True
7,Amazon's server logged the event with token asdfghjkl1234567890 at 10.10.10.10 on Friday.,"[Amazon, 's, server, logged, the, event, with, token, asdfghjkl1234567890, at, 10.10.10.10, on, Friday, .]","[B-org, O, O, O, O, O, O, O, B-Token, O, B-IP, O, B-tim, O]",True
8,The credential poiuytrewq_1234567890 was revoked from IP 172.20.14.5 during the summit in Berlin.,"[The, credential, poiuytrewq_1234567890, was, revoked, from, IP, 172.20.14.5, during, the, summit, in, Berlin, .]","[O, O, B-Token, O, O, O, O, B-IP, O, O, O, O, B-gpe, O]",True
9,Dr. Emily Clark verified the key 0987tyuio_pasdfghjkl from 2405:4800:100:101::1234 on Monday.,"[Dr., Emily, Clark, verified, the, key, 0987tyuio_pasdfghjkl, from, 2405:4800:100:101::1234, on, Monday, .]","[O, B-per, I-per, O, O, O, B-Token, O, B-IP, O, B-tim, O]",True


In [490]:
dataset_generated_6["Indicator"] = dataset_generated_6.apply(
    lambda row: len(row["tags"]) == len(row["tokens"]), axis=1
)

In [475]:
dataset_generated_7 = generate_ner_ds_2(num_row = 50,model_name = "gpt-4.1-mini")

In [491]:
dataset_generated_7

,sentence,tokens,tags,Indicator
0,"On Wednesday, the token a1b2c3_d4e5f6g7h8 accessed the server at 10.0.0.254 in Berlin.","[On, Wednesday, ,, the, token, a1b2c3_d4e5f6g7h8, accessed, the, server, at, 10.0.0.254, in, Berlin, .]","[O, B-tim, O, O, O, B-Token, O, O, O, O, B-IP, O, B-geo, O]",True
1,Elena verified credential z9x8w7v6u5t4r3q2p1 from 172.31.255.255 while visiting New York City.,"[Elena, verified, credential, z9x8w7v6u5t4r3q2p1, from, 172.31.255.255, while, visiting, New, York, City, .]","[B-per, O, O, B-Token, O, B-IP, O, O, B-geo, I-geo, I-geo, O]",True
2,The organization IBM sent the code 7h6g5f4d3s2a1q0w9e8r7t6y5u4i3o2p to 2607:f0d0:1002:51::4.,"[The, organization, IBM, sent, the, code, 7h6g5f4d3s2a1q0w9e8r7t6y5u4i3o2p, to, 2607:f0d0:1002:51::4, .]","[O, O, B-org, O, O, O, B-Token, O, B-IP, O]",True
3,"During the meeting in Paris, token 4r5t6y7u8i9o0p_asdfghjkl was authenticated from 192.0.2.146.","[During, the, meeting, in, Paris, ,, token, 4r5t6y7u8i9o0p_asdfghjkl, was, authenticated, from, 192.0.2.146, .]","[O, O, O, O, B-geo, O, O, B-Token, O, O, O, B-IP, O]",True
4,Michael Jordan accessed system token zxcvbnm_1234567890 from 172.217.14.206 last Friday.,"[Michael, Jordan, accessed, system, token, zxcvbnm_1234567890, from, 172.217.14.206, last, Friday, .]","[B-per, I-per, O, O, O, B-Token, O, B-IP, O, B-tim, O]",True
...,...,...,...,...
59,The key plmoknijbuhvygctfxrdzeswaq123456 was authenticated at 2607:f8b0:4005:805::200e in San Francisco.,"[The, key, plmoknijbuhvygctfxrdzeswaq123456, was, authenticated, at, 2607:f8b0:4005:805::200e, in, San, Francisco, .]","[O, O, B-Token, O, O, O, B-IP, O, B-geo, I-geo, O]",True
60,Michael Scott sent token qazwsxedcrfvtgbyhnujmikolp12345 to 198.51.100.60 last Friday.,"[Michael, Scott, sent, token, qazwsxedcrfvtgbyhnujmikolp12345, to, 198.51.100.60, last, Friday, .]","[B-per, I-per, O, O, B-Token, O, B-IP, O, B-tim, O]",True
61,The token asdfghjklqwertyuiopzxcvbnm987654 was verified in Rome at 172.16.0.150.,"[The, token, asdfghjklqwertyuiopzxcvbnm987654, was, verified, in, Rome, at, 172.16.0.150, .]","[O, O, B-Token, O, O, O, B-geo, O, B-IP, O]",True
62,Credential zxcvbnmlkjhgfdsaqwertyuiop098765 was authenticated by Apple at 203.0.113.50.,"[Credential, zxcvbnmlkjhgfdsaqwertyuiop098765, was, authenticated, by, Apple, at, 203.0.113.50, .]","[O, B-Token, O, O, O, B-org, O, B-IP, O]",True


In [492]:
dataset_generated_7["Indicator"] = dataset_generated_7.apply(
    lambda row: len(row["tags"]) == len(row["tokens"]), axis=1
)

In [480]:
dataset_generated_8 = generate_ner_ds_2(num_row = 50,model_name = "gpt-4.1-mini")

In [493]:
dataset_generated_8["Indicator"] = dataset_generated_8.apply(
    lambda row: len(row["tags"]) == len(row["tokens"]), axis=1
)

In [494]:
dataset_generated_8

,sentence,tokens,tags,Indicator
0,"On March 15th, the engineer with ID qw8x9zasdfghjklpoiuytrewq1234567_89-0 detected an anomaly from IP 172.16.254.1 near San Francisco.","[On, March, 15th, ,, the, engineer, with, ID, qw8x9zasdfghjklpoiuytrewq1234567_89-0, detected, an, anomaly, from, IP, 172.16.254.1, near, San, Francisco, .]","[O, B-tim, I-tim, O, O, O, O, O, B-Token, O, O, O, O, O, B-IP, O, B-geo, I-geo, O]",True
1,CEO Michael Thompson uploaded file xcvbnm_1234567-qwertyuiopasdfghjklzxcvbnm987654321 at 10.0.0.254 during the conference in Berlin.,"[CEO, Michael, Thompson, uploaded, file, xcvbnm_1234567-qwertyuiopasdfghjklzxcvbnm987654321, at, 10.0.0.254, during, the, conference, in, Berlin, .]","[O, B-per, I-per, O, O, B-Token, O, B-IP, O, O, O, O, B-gpe, O]",True
2,The server at IP 2607:f8b0:4005:805::200e received data from token ytrewq0987poiuy_1456-asdfghjklzxcvbnmqwertyuiop in New York.,"[The, server, at, IP, 2607:f8b0:4005:805::200e, received, data, from, token, ytrewq0987poiuy_1456-asdfghjklzxcvbnmqwertyuiop, in, New, York, .]","[O, O, O, O, B-IP, O, O, O, O, B-Token, O, B-geo, I-geo, O]",True
3,"During the meeting on 2023-11-28, Dr. Emily Carter used the access key hjklpoiuytrewq_1234567890-zxcvbnmasdfghjkl at IP 198.51.100.23 in Paris.","[During, the, meeting, on, 2023-11-28, ,, Dr., Emily, Carter, used, the, access, key, hjklpoiuytrewq_1234567890-zxcvbnmasdfghjkl, at, IP, 198.51.100.23, in, Paris, .]","[O, O, O, O, B-tim, O, O, B-per, I-per, O, O, O, O, B-Token, O, O, B-IP, O, B-gpe, O]",True
4,Amazon Web Services logged the token 1qaz2wsx3edc4rfv5tgb6yhn7ujm8iklop_0987654321 at IP 203.0.113.195 on Friday.,"[Amazon, Web, Services, logged, the, token, 1qaz2wsx3edc4rfv5tgb6yhn7ujm8iklop_0987654321, at, IP, 203.0.113.195, on, Friday, .]","[B-org, I-org, I-org, O, O, O, B-Token, O, O, B-IP, O, B-tim, O]",True
5,"At 08:45 AM, agent Sarah Connor accessed the system with token zxcvbnm_09876-asdfghjklpoiuytrewq1122334455 and IP 192.0.2.146 in Washington.","[At, 08:45, AM, ,, agent, Sarah, Connor, accessed, the, system, with, token, zxcvbnm_09876-asdfghjklpoiuytrewq1122334455, and, IP, 192.0.2.146, in, Washington, .]","[O, B-tim, I-tim, O, O, B-per, I-per, O, O, O, O, O, B-Token, O, O, B-IP, O, B-gpe, O]",True
6,IBM's server at IP 2404:6800:4008:0801::200e processed token 5tgb6yhn7ujm8iklop_1qaz2wsx3edc4rfv during the summit in Tokyo.,"[IBM, 's, server, at, IP, 2404:6800:4008:0801::200e, processed, token, 5tgb6yhn7ujm8iklop_1qaz2wsx3edc4rfv, during, the, summit, in, Tokyo, .]","[B-org, O, O, O, O, B-IP, O, O, B-Token, O, O, O, O, B-geo, O]",True
7,Lisa Wong submitted credentials abcdefghijklmnop_qrstuvwx-1234567890 to IP 198.18.0.42 while in Madrid last Monday.,"[Lisa, Wong, submitted, credentials, abcdefghijklmnop_qrstuvwx-1234567890, to, IP, 198.18.0.42, while, in, Madrid, last, Monday, .]","[B-per, I-per, O, O, B-Token, O, O, B-IP, O, O, B-geo, B-tim, I-tim, O]",True
8,The hacker used token qwertyuiopasdfghjklzxcvbnm0987654321_1234567890-abcd at the IP 10.10.10.10 during the New Year event in London.,"[The, hacker, used, token, qwertyuiopasdfghjklzxcvbnm0987654321_1234567890-abcd, at, the, IP, 10.10.10.10, during, the, New, Year, event, in, London, .]","[O, O, O, O, B-Token, O, O, O, B-IP, O, O, B-tim, I-tim, O, O, B-gpe, O]",True
9,Professor Alan Turing logged into the system using token asdfghjklqwertyuiopzxcvbnm1234567890_987654321 and IP 127.0.0.1 at 13:00 in Cambridge.,"[Professor, Alan, Turing, logged, into, the, system, using, token, asdfghjklqwertyuiopzxcvbnm1234567890_987654321, and, IP, 127.0.0.1, at, 13:00, in, Cambridge, .]","[O, B-per, I-per, O, O, O, O, O, O, B-Token, O, O, B-IP, O, B-tim, O, B-geo, O]",True


In [478]:
dataset_generated_9 = generate_ner_ds_2(num_row = 50,model_name = "gpt-4.1-mini")

In [496]:
dataset_generated_9.head()

,sentence,tokens,tags,Indicator
0,"Yesterday at 15:45, the engineer abcdefghijklmnopqrstu12345-xyz78_9 checked the server at 10.0.0.254 in San Francisco.","[Yesterday, at, 15:45, ,, the, engineer, abcdefghijklmnopqrstu12345-xyz78_9, checked, the, server, at, 10.0.0.254, in, San, Francisco, .]","[B-tim, O, B-tim, O, O, O, B-Token, O, O, O, O, B-IP, O, B-geo, I-geo, O]",True
1,Amazon secured the connection using token 4r5t6y7u8i9o0p_asdfghjkl-qwertyuiop at IP 172.16.254.1 in Seattle.,"[Amazon, secured, the, connection, using, token, 4r5t6y7u8i9o0p_asdfghjkl-qwertyuiop, at, IP, 172.16.254.1, in, Seattle, .]","[B-org, O, O, O, O, O, B-Token, O, O, B-IP, O, B-geo, O]",True
2,"On Monday morning, Dr. Sarah Connor entered the code zxcvbnm_asdfghjklqwertyuiop1234567890 at 192.168.0.100 in New Delhi.","[On, Monday, morning, ,, Dr., Sarah, Connor, entered, the, code, zxcvbnm_asdfghjklqwertyuiop1234567890, at, 192.168.0.100, in, New, Delhi, .]","[O, B-tim, I-tim, O, O, B-per, I-per, O, O, O, B-Token, O, B-IP, O, B-geo, I-geo, O]",True
3,Microsoft issued a new token key qwertyuiopasdfghjklzxcvbnm1234567890_0987-6543 at 2607:f0d0:1002:51::4 in Redmond.,"[Microsoft, issued, a, new, token, key, qwertyuiopasdfghjklzxcvbnm1234567890_0987-6543, at, 2607:f0d0:1002:51::4, in, Redmond, .]","[B-org, O, O, O, O, O, B-Token, O, B-IP, O, B-geo, O]",True
4,"At 23:59, agent 9w8e7r6t5y4u3i2o1p0-lkjhgfdsazxcvbnm checked IP 8.8.4.4 during the conference in Berlin.","[At, 23:59, ,, agent, 9w8e7r6t5y4u3i2o1p0-lkjhgfdsazxcvbnm, checked, IP, 8.8.4.4, during, the, conference, in, Berlin, .]","[O, B-tim, O, O, B-Token, O, O, B-IP, O, O, O, O, B-geo, O]",True


In [495]:
dataset_generated_9["Indicator"] = dataset_generated_9.apply(
    lambda row: len(row["tags"]) == len(row["tokens"]), axis=1
)

In [299]:
dataset_generated_10 = generate_ner_ds_2(num_row = 50,model_name = "gpt-4.1-mini")

In [300]:
dataset_generated_10["Indicator"] = dataset_generated_10.apply(
    lambda row: len(row["tags"]) == len(row["tokens"]), axis=1
)

In [ ]:
dataset_generated_10

In [302]:
dataset_generated_11 = generate_ner_ds_2(num_row = 50,model_name = "gpt-4.1-mini")

In [303]:
dataset_generated_11["Indicator"] = dataset_generated_11.apply(
    lambda row: len(row["tags"]) == len(row["tokens"]), axis=1
)

In [304]:
dataset_generated_12 = generate_ner_ds_2(num_row = 50,model_name = "gpt-4.1-mini")

In [305]:
dataset_generated_12["Indicator"] = dataset_generated_12.apply(
    lambda row: len(row["tags"]) == len(row["tokens"]), axis=1
)

In [306]:
dataset_generated_13 = generate_ner_ds_2(num_row = 50,model_name = "gpt-4.1-mini")

In [307]:
dataset_generated_13["Indicator"] = dataset_generated_13.apply(
    lambda row: len(row["tags"]) == len(row["tokens"]), axis=1
)

In [ ]:
dataset_generated_13

In [309]:
dataset_generated_14 = generate_ner_ds_2(num_row = 50,model_name = "gpt-4.1-mini")

In [310]:
dataset_generated_14["Indicator"] = dataset_generated_14.apply(
    lambda row: len(row["tags"]) == len(row["tokens"]), axis=1
)

In [311]:
dataset_generated_15 = generate_ner_ds_2(num_row = 50,model_name = "gpt-4.1-mini")

In [312]:
dataset_generated_15["Indicator"] = dataset_generated_15.apply(
    lambda row: len(row["tags"]) == len(row["tokens"]), axis=1
)

In [339]:
dataset_generated_16 = generate_ner_ds_2(num_row = 50,model_name = "gpt-4.1-mini")

In [ ]:
dataset_generated_16

In [341]:
dataset_generated_16["Indicator"] = dataset_generated_16.apply(
    lambda row: len(row["tags"]) == len(row["tokens"]), axis=1
)

In [342]:
dataset_generated_17 = generate_ner_ds_2(num_row = 50,model_name = "gpt-4.1-mini")

In [343]:
dataset_generated_17["Indicator"] = dataset_generated_17.apply(
    lambda row: len(row["tags"]) == len(row["tokens"]), axis=1
)

In [344]:
dataset_generated_18 = generate_ner_ds_2(num_row = 50,model_name = "gpt-4.1-mini")

In [345]:
dataset_generated_18["Indicator"] = dataset_generated_18.apply(
    lambda row: len(row["tags"]) == len(row["tokens"]), axis=1
)

In [346]:
dataset_generated_19 = generate_ner_ds_2(num_row = 50,model_name = "gpt-4.1-mini")

In [347]:
dataset_generated_19["Indicator"] = dataset_generated_19.apply(
    lambda row: len(row["tags"]) == len(row["tokens"]), axis=1
)

In [348]:
dataset_generated_20 = generate_ner_ds_2(num_row = 50,model_name = "gpt-4.1-mini")

In [357]:
dataset_generated_21 = generate_ner_ds_2(num_row = 50,model_name = "gpt-4.1-mini")

In [359]:
dataset_generated_21["Indicator"] = dataset_generated_21.apply(
    lambda row: len(row["tags"]) == len(row["tokens"]), axis=1
)

In [360]:
dataset_generated_22 = generate_ner_ds_2(num_row = 50,model_name = "gpt-4.1-mini")

In [361]:
dataset_generated_22["Indicator"] = dataset_generated_22.apply(
    lambda row: len(row["tags"]) == len(row["tokens"]), axis=1
)

In [362]:
dataset_generated_23 = generate_ner_ds_2(num_row = 50,model_name = "gpt-4.1-mini")

In [363]:
dataset_generated_24 = generate_ner_ds_2(num_row = 50,model_name = "gpt-4.1-mini")

In [364]:
dataset_generated_25 = generate_ner_ds_2(num_row = 50,model_name = "gpt-4.1-mini")

In [391]:
dataset_generated_26 = generate_ner_ds_2(num_row = 50,model_name = "gpt-4.1-mini")

In [ ]:
dataset_generated_27 = generate_ner_ds_2(num_row = 50,model_name = "gpt-4.1-mini")

In [497]:

dataset = [globals()[f"dataset_generated_{i}"] for i in range(1, 11)]

In [498]:
train_data = pd.concat(dataset)

In [201]:
train_data.to_csv("train_data_1.csv")

In [424]:
train_data.to_csv("train_data_2.csv")

In [510]:
train_data.to_csv("train_data_3.csv", index = False)

In [ ]:
train_data

In [499]:
def convert_bilu_to_spacy_format(sentence, tokens, tags):
    doc = nlp.make_doc(sentence)

    if len(tokens) != len(tags) or len(tokens) != len(doc):
        raise ValueError(f"Length mismatch: {len(tokens)} tokens, {len(tags)} tags, {len(doc)} spaCy tokens.")

    entities = []
    i = 0
    while i < len(tags):
        tag = tags[i]

        if tag == "O":
            i += 1
            continue

        prefix, label = tag.split("-", 1)
        label = label.upper()

        if prefix == "B":  
            start_char = doc[i].idx
            j = i + 1

            while j < len(tags):
                next_tag = tags[j]
                if next_tag.startswith("I-") and next_tag[2:] == label:
                    j += 1
                elif next_tag.startswith("L-") and next_tag[2:] == label:
                    end_char = doc[j].idx + len(doc[j])
                    entities.append((start_char, end_char, label))
                    break
                else:
                    # Unexpected end: fallback to just B- as single token
                    end_char = doc[i].idx + len(doc[i])
                    entities.append((start_char, end_char, label))
                    j = i 
                    break
            i = j + 1  # move past L- tag

        else:
            # Skip I- or L- not following a B-
            i += 1

    return (sentence, {"entities": entities})


In [500]:
from IPython.core.debugger import set_trace

spacy_data = []
for idx, row in train_data.iterrows():
    sentence = row["sentence"]
    tokens = row["tokens"]
    tags = row["tags"]
    try:
        spacy_format = convert_bilu_to_spacy_format(sentence, tokens, tags)
        if spacy_format is None:
            print(f"Row {idx}: Returned None")
        spacy_data.append(spacy_format)
    except Exception as e:
        print(f"Failed to convert row {idx}: {sentence[:50]}... — {e}")
        

Failed to convert row 1: At 09:15, Dr. Helena Schmidt sent the encrypted me... — Length mismatch: 17 tokens, 17 tags, 19 spaCy tokens.
Failed to convert row 2: The conference in New Delhi started at 2023-11-25T... — Length mismatch: 15 tokens, 15 tags, 21 spaCy tokens.
Failed to convert row 3: Jessica and Michael sent the file qw987asdfghjklzx... — Length mismatch: 12 tokens, 12 tags, 16 spaCy tokens.
Failed to convert row 4: Google updated their logs with token r4nd0m_5tr1ng... — Length mismatch: 17 tokens, 17 tags, 19 spaCy tokens.
Failed to convert row 5: On Monday, the access code 1234567890abcdef-ghijkl... — Length mismatch: 16 tokens, 16 tags, 18 spaCy tokens.
Failed to convert row 9: The update code 5f6e7d8c9b0a1fghijklmno_pqrstuvwx-... — Length mismatch: 17 tokens, 17 tags, 19 spaCy tokens.
Failed to convert row 1: At 09:00 AM, CEO Alice Johnson uploaded the file z... — Length mismatch: 19 tokens, 19 tags, 21 spaCy tokens.
Failed to convert row 2: The device x1y2z3a4b5c6d7e8f9g

In [501]:
len(spacy_data)

369

In [78]:
label2id = {"CARDINAL": 1,
 'DATE': 2,
 'LANGUAGE': 3,
 'LOC': 4,
 'MONEY': 5,
 'ORDINAL': 6,
 'ORG': 7,
  'QUANTITY':8,
  'WORK_OF_ART': 9,
  'PER': 10,
 'MISC': 0}


# Load Spacy

In [502]:
nlp = spacy.load("en_core_web_lg")

In [ ]:
("What is the price of 10 bananas?", {"entities": [(21, 23, "QUANTITY"), (24, 31, "PRODUCT")]})

In [66]:
import spacy
from spacy.training import Example
import random
from spacy.util import minibatch
from spacy.training import offsets_to_biluo_tags

In [ ]:
('Dr. Emily Carter from the University of Cambridge accessed the system using token XJ93ks8df7aY1zq and IP 192.168.1.101 yesterday.',
  {'entities': [(4, 16, 'PER'),
    (26, 36, 'ORG'),
    (40, 49, 'GEO'),
    (119, 128, 'TIM')]}),

In [503]:

# Add or get NER pipe
if "ner" not in nlp.pipe_names:
    ner = nlp.add_pipe("ner", last=True)
else:
    ner = nlp.get_pipe("ner")



In [504]:
ner.labels

('CARDINAL',
 'DATE',
 'EVENT',
 'FAC',
 'GPE',
 'LANGUAGE',
 'LAW',
 'LOC',
 'MONEY',
 'NORP',
 'ORDINAL',
 'ORG',
 'PERCENT',
 'PERSON',
 'PRODUCT',
 'QUANTITY',
 'TIME',
 'WORK_OF_ART')

In [505]:
ner.add_label("Token")

1

In [506]:
ner.add_label("IP")

1

In [402]:
ner.labels

('CARDINAL',
 'DATE',
 'EVENT',
 'FAC',
 'GPE',
 'IP',
 'LANGUAGE',
 'LAW',
 'LOC',
 'MONEY',
 'NORP',
 'ORDINAL',
 'ORG',
 'PERCENT',
 'PERSON',
 'PRODUCT',
 'QUANTITY',
 'TIME',
 'Token',
 'WORK_OF_ART')

In [185]:
spacy_data

[('During the conference in New York, the token h4jd93kslqw8EYTU12 was verified from IP 192.168.1.15.',
  {'entities': [(25, 28, 'GEO'), (45, 63, 'TOKEN'), (85, 97, 'IP')]}),
 ('Amazon hired a new employee with token ZXCVBNM123456qwerty and IP 2001:0db8:85a3:0000:0000:8a2e:0370:7334.',
  {'entities': [(0, 6, 'ORG'), (39, 58, 'TOKEN'), (66, 105, 'IP')]}),
 ('On March 5th, Dr. Emily Clark accessed the system using token a1b2c3d4e5f6g7h8i9j0 and IP 10.0.0.254.',
  {'entities': [(3, 8, 'TIM'),
    (14, 17, 'PER'),
    (62, 82, 'TOKEN'),
    (90, 100, 'IP')]}),
 ('Microsoft announced on January 12th that token QWERTYUIOPASDFGHJKL was compromised at IP 172.16.254.1.',
  {'entities': [(0, 9, 'ORG'),
    (23, 30, 'TIM'),
    (47, 66, 'TOKEN'),
    (89, 101, 'IP')]}),
 ('The summit held in Berlin featured token ZX12345ABCD67890EFGH and IP address 8.8.8.8.',
  {'entities': [(19, 25, 'GEO'), (41, 61, 'TOKEN'), (77, 84, 'IP')]}),
 ('President John Doe logged in using token TYU89JKL34MNOPQ12 and IP

In [507]:
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
with nlp.disable_pipes(*other_pipes):
    optimizer = nlp.create_optimizer()  # Default Adam optimizer
    epochs = 30
    for epoch in range(epochs):
        random.shuffle(spacy_data)
        losses = {}
        batches = minibatch(spacy_data, size=8)  # Increased batch size for en_core_web_sm
        for batch in batches:
            examples = []
            for text, annotations in batch:
                doc = nlp.make_doc(text)
                example = Example.from_dict(doc, annotations)  # Create Example object
                examples.append(example)
            nlp.update(examples, drop=0.2, sgd=optimizer, losses=losses)
        print(f"Epoch {epoch+1}, Losses: {losses}")

Epoch 1, Losses: {'ner': np.float32(1269.0942)}
Epoch 2, Losses: {'ner': np.float32(246.31627)}
Epoch 3, Losses: {'ner': np.float32(139.2854)}
Epoch 4, Losses: {'ner': np.float32(104.77164)}
Epoch 5, Losses: {'ner': np.float32(94.52413)}
Epoch 6, Losses: {'ner': np.float32(83.91225)}
Epoch 7, Losses: {'ner': np.float32(72.005806)}
Epoch 8, Losses: {'ner': np.float32(72.07907)}
Epoch 9, Losses: {'ner': np.float32(49.920387)}
Epoch 10, Losses: {'ner': np.float32(60.354675)}
Epoch 11, Losses: {'ner': np.float32(47.58746)}
Epoch 12, Losses: {'ner': np.float32(40.05519)}
Epoch 13, Losses: {'ner': np.float32(36.6704)}
Epoch 14, Losses: {'ner': np.float32(28.56426)}
Epoch 15, Losses: {'ner': np.float32(32.423943)}
Epoch 16, Losses: {'ner': np.float32(29.017216)}
Epoch 17, Losses: {'ner': np.float32(14.961398)}
Epoch 18, Losses: {'ner': np.float32(22.66567)}
Epoch 19, Losses: {'ner': np.float32(26.296474)}
Epoch 20, Losses: {'ner': np.float32(25.366358)}
Epoch 21, Losses: {'ner': np.float32(26

In [191]:
nlp.to_disk("/Users/senghok/Documents/Internship_2A/code/gen_data/Spacy_model")

In [508]:
test_sentences = [
    'here is your token chhay dgpJ6SMGbuL9ZLstl4RhL5vTT3BlbkFJ_jAyYC2dk3AaP5Qo8MQwilk7viB0wuKZvfMHWm0ZtbjcTt1G1JRr9THEQ9Ee7nZz71N8a5AigA',
    'The secure code dgpJ6SMGbuL9ZLstl4RhL5vTT3BlbkFJ_jAyYC2dk3AaP5Qo8MQwilk7viB0wuKZvfMHWm0ZtbjcTt1G1JRr9THEQ9Ee7nZz71N8a5AigA was generated at 10:45 AM by Dr. Emily Smith.',
    'Token zdgpJ6SMGbuL9ZLstl4RhL5vTT3BlbkFJ_jAyYC2dk3AaP5Qo8MQwilk7viB0wuKZvfMHWm0ZtbjcTt1G1JRr9THEQ9Ee7nZz71N8a5AigA was flagged by the firewall at 192.168.1.1.',
    'On Monday, tech lead Bob Lee shared token QWErty123$%^ via 2001:db8::1.',
    "Customer's 7gH*9kl@2qW was used to connect from IP 172.20.10.5.",
    'System issued A1B2C3D4 at 03:00 for login from 2606:4700:4700::1111.',
    'During audit, the token secure!!Token++ was seen at 127.0.0.1.',
    'Log shows Alice sent dgpJ6SMGbuL9ZLstl4RhL5vTT3BlbkFJ_jAyYC2dk3AaP5Qo8MQwilk7viB0wuKZvfMHWm0ZtbjcTt1G1JRr9THEQ9Ee7nZz71N8a5AigA from IP 10.0.0.42 at 18:20.',
]

for sentence in test_sentences:
    doc = nlp(sentence)
    print(f"\nText: {sentence}")
    print("Entities:", [(ent.text, ent.label_) for ent in doc.ents])



Text: here is your token chhay dgpJ6SMGbuL9ZLstl4RhL5vTT3BlbkFJ_jAyYC2dk3AaP5Qo8MQwilk7viB0wuKZvfMHWm0ZtbjcTt1G1JRr9THEQ9Ee7nZz71N8a5AigA
Entities: [('chhay', 'TOKEN')]

Text: The secure code dgpJ6SMGbuL9ZLstl4RhL5vTT3BlbkFJ_jAyYC2dk3AaP5Qo8MQwilk7viB0wuKZvfMHWm0ZtbjcTt1G1JRr9THEQ9Ee7nZz71N8a5AigA was generated at 10:45 AM by Dr. Emily Smith.
Entities: [('dgpJ6SMGbuL9ZLstl4RhL5vTT3BlbkFJ_jAyYC2dk3AaP5Qo8MQwilk7viB0wuKZvfMHWm0ZtbjcTt1G1JRr9THEQ9Ee7nZz71N8a5AigA', 'TOKEN'), ('10:45', 'TIM'), ('Dr.', 'PER'), ('Emily', 'PER')]

Text: Token zdgpJ6SMGbuL9ZLstl4RhL5vTT3BlbkFJ_jAyYC2dk3AaP5Qo8MQwilk7viB0wuKZvfMHWm0ZtbjcTt1G1JRr9THEQ9Ee7nZz71N8a5AigA was flagged by the firewall at 192.168.1.1.
Entities: [('zdgpJ6SMGbuL9ZLstl4RhL5vTT3BlbkFJ_jAyYC2dk3AaP5Qo8MQwilk7viB0wuKZvfMHWm0ZtbjcTt1G1JRr9THEQ9Ee7nZz71N8a5AigA', 'TOKEN'), ('192.168.1.1', 'IP')]

Text: On Monday, tech lead Bob Lee shared token QWErty123$%^ via 2001:db8::1.
Entities: [('Monday', 'TIM'), ('Bob', 'PER'), ('QWErty123$%^', 'TOKEN'

In [192]:
nlp = spacy.load("Spacy_model")

In [193]:
nlp.get_pipe("ner").labels

('CARDINAL',
 'DATE',
 'EVENT',
 'FAC',
 'GEO',
 'GPE',
 'IP',
 'LANGUAGE',
 'LAW',
 'LOC',
 'MONEY',
 'NORP',
 'ORDINAL',
 'ORG',
 'PER',
 'PERCENT',
 'PERSON',
 'PRODUCT',
 'QUANTITY',
 'TIM',
 'TIME',
 'TOKEN',
 'Token',
 'WORK_OF_ART')